In [30]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import math
from sklearn.cluster import KMeans
from transform import four_point_transform
%matplotlib inline

In [31]:
data_path = "/home/avk/study/1C/Diplom/NewData/samples/"

In [32]:
for file_num, filename in enumerate(os.listdir(data_path)):
    full_file_path = os.path.join(data_path, filename)
    if file_num == 5:
#     if file_num == 2:
        break
file_num

2

In [33]:
file_num_to_save = 1

def show_image(image, title="", mode="show"):
    """
    image: image, which was read by opencv
    """
#     plt.figure(figsize=(10, 20))
#     plt.title(title)
#     image_to_show = image
#     if len(image.shape) > 2:
#         image_to_show = image[:, :, ::-1]
#     plt.imshow(image_to_show)
    image_to_show = image
    if max(image.shape) > 1000:
        image_to_show = cv2.resize(image, (720, 960), interpolation=cv2.INTER_CUBIC)
    if mode == "show":
        cv2.imshow(title, image_to_show)
        cv2.waitKey(0)
    elif mode == "save":
        global file_num_to_save
        cv2.imwrite(str(file_num_to_save) + ".jpg", image_to_show)
        file_num_to_save += 1

In [34]:
initial_image = cv2.imread(full_file_path)
# plt.figure(figsize=(10, 20))
# plt.imshow(initial_image)
initial_image.shape
show_image(initial_image)

# Уменьшение изображения

In [35]:
SHORT_SIDE_SMALL_LEN = 300
initial_small_side_len = sorted(initial_image.shape[:2])[0]
resizing_ratio = initial_small_side_len / SHORT_SIDE_SMALL_LEN

resized_image = cv2.resize(initial_image, None, fx=1 / resizing_ratio, 
                           fy=1 / resizing_ratio, interpolation=cv2.INTER_CUBIC)
                           
show_image(resized_image)
resized_image.shape

(401, 300, 3)

# Размытие текста

In [36]:
def blur_image(image, blur, kernel_size, times=1):
    resulted_image = image
    for i in range(times):
        try:
            resulted_image = blur(resulted_image, kernel_size)
        except:
            resulted_image = blur(resulted_image, (kernel_size, kernel_size), 0)
    return resulted_image

In [37]:
# for times in range(1, 8):
#     smoothed_image = blur_image(resized_image, cv2.medianBlur, 5, times)
#     show_image(smoothed_image, str(times))

In [38]:
smoothed_image = blur_image(resized_image, cv2.medianBlur, 5, 3)
show_image(smoothed_image)
# smoothed_image = blur_image(smoothed_image, cv2.medianBlur, 3, 3)
# show_image(smoothed_image)

# Выделение границ

In [42]:
edges = cv2.Canny(smoothed_image, 10, 50, apertureSize=3)
show_image(edges)

In [43]:
hough_lines = cv2.HoughLines(edges, 1, np.pi / 180, 100, None, 0, 0)
# show_image(hough_lines)
len(hough_lines)

4

In [44]:
cdst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

In [45]:
# show_image(cdst)

In [46]:
# Draw the lines
if hough_lines is not None:
    for i in range(0, len(hough_lines)):
        rho = hough_lines[i][0][0]
        theta = hough_lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv2.line(cdst, pt1, pt2, (0,0,255), 1, cv2.LINE_AA)

In [47]:
show_image(cdst)

In [48]:
from collections import defaultdict
def segment_by_angle_kmeans(lines, k=2, **kwargs):
    """Groups lines based on angle with k-means.

    Uses k-means on the coordinates of the angle on the unit circle 
    to segment `k` angles inside `lines`.
    """

    # Define criteria = (type, max_iter, epsilon)
    default_criteria_type = cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER
    criteria = kwargs.get('criteria', (default_criteria_type, 10, 1.0))
    flags = kwargs.get('flags', cv2.KMEANS_RANDOM_CENTERS)
    attempts = kwargs.get('attempts', 10)

    # returns angles in [0, pi] in radians
    angles = np.array([line[0][1] for line in lines])
    # multiply the angles by two and find coordinates of that angle
    pts = np.array([[np.cos(2*angle), np.sin(2*angle)]
                    for angle in angles], dtype=np.float32)

    # run kmeans on the coords
    labels, centers = cv2.kmeans(pts, k, None, criteria, attempts, flags)[1:]
    labels = labels.reshape(-1)  # transpose to row vec

    # segment lines based on their kmeans label
    segmented = defaultdict(list)
    for i, line in zip(range(len(lines)), lines):
        segmented[labels[i]].append(line)
    segmented = list(segmented.values())
    return segmented

In [49]:
segmented = segment_by_angle_kmeans(hough_lines)

In [50]:
segmented

[[array([[355.       ,   1.5707964]], dtype=float32),
  array([[33.      ,  1.553343]], dtype=float32),
  array([[30.       ,  1.5707964]], dtype=float32)],
 [array([[3.5000000e+01, 1.7453292e-02]], dtype=float32)]]

In [51]:
def intersection(line1, line2):
    """Finds the intersection of two lines given in Hesse normal form.

    Returns closest integer pixel locations.
    See https://stackoverflow.com/a/383527/5087436
    """
    rho1, theta1 = line1[0]
    rho2, theta2 = line2[0]
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]


def segmented_intersections(lines):
    """Finds the intersections between groups of lines."""

    intersections = []
    for i, group in enumerate(lines[:-1]):
        for next_group in lines[i+1:]:
            for line1 in group:
                for line2 in next_group:
                    intersections.append(intersection(line1, line2)) 

    return intersections

In [52]:
intersections = segmented_intersections(segmented)

In [53]:
len(intersections)

3

In [54]:
points = []
for point in intersections:
    coordinates = tuple(point[0])
    points.append(coordinates)
#     print(coordinates)
    cv2.circle(cdst, coordinates, 3, (0, 255, 0), -1)
show_image(cdst)

In [55]:
np.array(points)

array([[ 29, 355],
       [ 34,  32],
       [ 34,  30]])

In [56]:
angles = KMeans(n_clusters=4).fit(np.array(points)).cluster_centers_

ValueError: n_samples=3 should be >= n_clusters=4

In [25]:
for angle in angles:
    angle = tuple(np.array(angle, dtype=int))
    print(angle)
    cv2.circle(cdst, angle, 3, (255, 0, 0), -1)
show_image(cdst)

(251, 348)
(236, 70)
(41, 345)
(55, 70)


In [26]:
angles  = np.array(angles * resizing_ratio, dtype=int)
angles

array([[2523, 3498],
       [2372,  703],
       [ 412, 3468],
       [ 552,  703]])

In [27]:
warped = four_point_transform(initial_image, angles)
show_image(warped)